In [1]:
using Pkg
Pkg.activate(@__DIR__)

  Activating project at `/global/u1/b/blaschke/DistributedStreams.jl`


In [2]:
project_path = @__DIR__

"/global/u1/b/blaschke/DistributedStreams.jl"

In [3]:
using Distributed, ClusterManagers, Sockets, Chain, Dates, TimeZones, JLD2, FileIO, Plots

In [4]:
epoch_utc(dt) = @chain dt begin
    ZonedDateTime(_, tz"America/Los_Angeles")
    astimezone(_, tz"UTC")
    DateTime(_)
    datetime2unix(_)
    floor(Int, _)
end

epoch_utc (generic function with 1 method)

In [5]:
# function request_worker()
#     @async begin
#         addprocs(
#             SlurmManager(32),
#             N="1", constraint="gpu", qos="regular", time="00:10:00",
#             account="nstaff"
#         )
#     end
# end
# function request_worker(N, C, qos, t, em)
#     task = @async run(
#         `srun -n $(N) -A "nstaff" -C $(C) -q $(qos) -t $(t) sh -c $(ClusterManagers.get_connect_cmd(em))`
#     )
#     return task
# end

In [6]:
em = ElasticManager(addr=:auto, topology=:master_worker, port=9009)

ElasticManager:
  Active workers : []
  Number of workers to be added  : 0
  Terminated workers : []
  Worker connect command : 
    /global/common/software/nersc/n9/julia/1.9.4/bin/julia --project=/global/u1/b/blaschke/DistributedStreams.jl/Project.toml -e 'using ClusterManagers; ClusterManagers.elastic_worker("ULWmH53Dc65s8DpY","10.249.0.15",9009)'

In [7]:
# worker_log = Dict{Int64, Vector{Int64}}()
worker_log = Dict{Int64, Int64}()

Dict{Int64, Int64}()

In [8]:
stop_monitor = false

tm = @async while true
    if stop_monitor
        println("global interrupt=$(stop_monitor) => stopping task")
        break
    end
    worker_log[epoch_utc(now())] = nworkers()
    sleep(10)
end

Task (runnable) @0x00007f223a96fd00

In [9]:
worker_log

Dict{Int64, Int64} with 1 entry:
  1713290714 => 1

In [10]:
n_jobs = 128

N   = 1
n   = N*32
C   = "gpu"
qos = "regular"
t   = "00:05:00"


# jobs = Task[]
# for i in 1:n_jobs
#     push!(jobs, request_worker(N, C, qos, t, em))
#     sleep(10)
# end

"00:05:00"

In [11]:
job_script = """#!/bin/bash
#SBATCH -q $(qos)
#SBATCH -A nstaff
#SBATCH -N $(N)
#SBATCH -C $(C)
#SBATCH -t $(t)
#SBATCH -J julia-em-test

srun -n $(n) $(ClusterManagers.get_connect_cmd(em))
"""
cd(joinpath(project_path, "jobs"))
write("em_launcher.sh", job_script)
cd(project_path)

In [12]:
job_script = """#!/bin/bash
#SBATCH -q $(qos)
#SBATCH -A nstaff
#SBATCH -N $(N*64)
#SBATCH -C $(C)
#SBATCH -t $(t)
#SBATCH -J julia-em-test

srun -n $(n*64) $(ClusterManagers.get_connect_cmd(em))
"""
cd(joinpath(project_path, "jobs"))
write("em_launcher_big.sh", job_script)
cd(project_path)

In [13]:
cd(joinpath(project_path, "jobs"))
run(`sbatch em_launcher_big.sh`)
cd(project_path)

Submitted batch job 24495148


In [27]:
stop_submit = false

ts = @async for i in 1:n_jobs
    if stop_submit
        println("global interrupt=$(stop_submit) => stopping task")
        break
    end
    cd(joinpath(project_path, "jobs"))
    run(`sbatch em_launcher.sh`)
    cd(project_path)
    sleep(1)
end

Task (runnable) @0x00007f223abaf210

Submitted batch job 24497483
Submitted batch job 24497492
Submitted batch job 24497502
Submitted batch job 24497515
Submitted batch job 24497531
Submitted batch job 24497538
Submitted batch job 24497549
Submitted batch job 24497559
Submitted batch job 24497564
Submitted batch job 24497566
Submitted batch job 24497568
Submitted batch job 24497576
Submitted batch job 24497578
Submitted batch job 24497581
Submitted batch job 24497584
Submitted batch job 24497600
Submitted batch job 24497614
Submitted batch job 24497623
Submitted batch job 24497630
Submitted batch job 24497636
Submitted batch job 24497649
Submitted batch job 24497651
Submitted batch job 24497661
Submitted batch job 24497683
Submitted batch job 24497691
Submitted batch job 24497698
Submitted batch job 24497719
Submitted batch job 24497726
Submitted batch job 24497750


In [35]:
istaskdone(ts)

true

In [23]:
stop_submit=true

true

Submitted batch job 24492093
global interrupt=true => stopping task


In [44]:
worker_log

Dict{Int64, Int64} with 1797 entries:
  1713294734 => 1
  1713297004 => 1
  1713299215 => 1
  1713301635 => 1
  1713292884 => 1
  1713300565 => 1
  1713303845 => 1
  1713292744 => 1
  1713292544 => 1
  1713303855 => 1
  1713300745 => 1
  1713291524 => 1
  1713294984 => 1
  1713292374 => 1
  1713307626 => 1
  1713295454 => 1
  1713300215 => 1
  1713291604 => 1
  1713308326 => 32
  1713294424 => 1
  1713297714 => 1
  1713302755 => 1
  1713292304 => 1
  1713296274 => 1
  1713303955 => 1
  ⋮          => ⋮

In [42]:
nworkers()

1

In [22]:
stop_monitor = true

true

Submitted batch job 24492080
Submitted batch job 24492087
global interrupt=true => stopping task


In [40]:
data_set = 9
t_save = @async for i in 1:12
    save("worker_log_$(data_set)_$(i).jld2",
        "worker_log", worker_log,
        "n_jobs", n_jobs,
        "N", N,
        "C", C,
        "qos", qos,
        "t", t
    )
    sleep(3600)
end

Task (runnable) @0x00007f21df138010

In [45]:
# final_data_set = 9
# final_sub_set = 2
# save("worker_log_$(final_data_set)b_$(final_sub_set).jld2",
#     "worker_log", worker_log,
#     "n_jobs", n_jobs,
#     "N", N,
#     "C", C,
#     "qos", qos,
#     "t", t
# )

      From worker 2098:	
      From worker 2098:	[1576826] signal (15): Terminated
      From worker 2103:	
      From worker 2103:	[1576876] signal (15): Terminated
      From worker 2103:	in expression starting at none:1
      From worker 2103:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2102:	
      From worker 2102:	[1576856] signal (15): Terminated
      From worker 2102:	in expression starting at none:1
      From worker 2102:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2087:	
      From worker 2087:	[1576868] signal (15): Terminated
      From worker 2087:	in expression starting at none:1
      From worker 2087:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2088:	
      From worker 2088:	[1576870] signal (15): Terminated
      From worker 2088:	in expression starting at none:1
      From worker 2088:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2091:	
      From worker 2091:	[1576858] signal (15): Termi

Worker 2083 terminated.
Unhandled Task ERROR: EOFError: read end of file
Stacktrace:
 [1] (::Base.var"#wait_locked#715")(s::TCPSocket, buf::IOBuffer, nb::Int64)
   @ Base ./stream.jl:947
 [2] unsafe_read(s::TCPSocket, p::Ptr{UInt8}, nb::UInt64)
   @ Base ./stream.jl:955
 [3] unsafe_read
   @ ./io.jl:761 [inlined]
 [4] unsafe_read(s::TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
   @ Base ./io.jl:760
 [5] read!
   @ ./io.jl:762 [inlined]
 [6] deserialize_hdr_raw
   @ /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/messages.jl:167 [inlined]
 [7] message_handler_loop(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
   @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_messages.jl:172
 [8] process_tcp_streams(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
   @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_mess

      From worker 2140:	
      From worker 2140:	[1839588] signal (15): Terminated
      From worker 2140:	in expression starting at none:1
      From worker 2123:	
      From worker 2123:	[1839598] signal (15): Terminated
      From worker 2123:	in expression starting at none:1
      From worker 2123:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2137:	
      From worker 2137:	[1839596] signal (15): Terminated
      From worker 2137:	in expression starting at none:1
      From worker 2137:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2118:	
      From worker 2118:	[1839594] signal (15): Terminated
      From worker 2118:	in expression starting at none:1
      From worker 2118:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2144:	
      From worker 2144:	[1839590] signal (15): Terminated
      From worker 2144:	in expression starting at none:1
      From worker 2144:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 21

Worker 2119 terminated.Unhandled Task ERROR: IOError: read: connection reset by peer (ECONNRESET)
Stacktrace:
  [1] wait_readnb(x::TCPSocket, nb::Int64)
    @ Base ./stream.jl:410
  [2] (::Base.var"#wait_locked#715")(s::TCPSocket, buf::IOBuffer, nb::Int64)
    @ Base ./stream.jl:949
  [3] unsafe_read(s::TCPSocket, p::Ptr{UInt8}, nb::UInt64)
    @ Base ./stream.jl:955
  [4] unsafe_read
    @ ./io.jl:761 [inlined]
  [5] unsafe_read(s::TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
    @ Base ./io.jl:760
  [6] read!
    @ ./io.jl:762 [inlined]
  [7] deserialize_hdr_raw
    @ /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/messages.jl:167 [inlined]
  [8] message_handler_loop(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
    @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_messages.jl:172
  [9] process_tcp_streams(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
    

      From worker 2174:	
      From worker 2174:	[259035] signal (15): Terminated
      From worker 2171:	
      From worker 2171:	[259123] signal (15): Terminated
      From worker 2171:	in expression starting at none:1
      From worker 2171:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2154:	
      From worker 2154:	[259126] signal (15): Terminated
      From worker 2154:	in expression starting at none:1
      From worker 2154:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2166:	
      From worker 2166:	[259036] signal (15): Terminated
      From worker 2166:	in expression starting at none:1
      From worker 2161:	
      From worker 2161:	[259042] signal (15): Terminated
      From worker 2161:	in expression starting at none:1
      From worker 2169:	
      From worker 2169:	[259048] signal (15): Terminated
      From worker 2169:	in expression starting at none:1
      From worker 2158:	
      From worker 2158:	[259051] signal (15): Terminated
 

...

Worker 2204 terminated.Unhandled Task ERROR: IOError: read: connection reset by peer (ECONNRESET)
Stacktrace:
  [1] wait_readnb(x::TCPSocket, nb::Int64)
    @ Base ./stream.jl:410
  [2] (::Base.var"#wait_locked#715")(s::TCPSocket, buf::IOBuffer, nb::Int64)
    @ Base ./stream.jl:949
  [3] unsafe_read(s::TCPSocket, p::Ptr{UInt8}, nb::UInt64)
    @ Base ./stream.jl:955
  [4] unsafe_read
    @ ./io.jl:761 [inlined]
  [5] unsafe_read(s::TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
    @ Base ./io.jl:760
  [6] read!
    @ ./io.jl:762 [inlined]
  [7] deserialize_hdr_raw
    @ /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/messages.jl:167 [inlined]
  [8] message_handler_loop(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
    @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_messages.jl:172
  [9] process_tcp_streams(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
    

      From worker 2305:	
      From worker 2305:	[1588089] signal (15): Terminated
      From worker 2305:	in expression starting at none:1
      From worker 2304:	
      From worker 2304:	[1588092] signal (15): Terminated
      From worker 2304:	in expression starting at none:1
      From worker 2282:	
      From worker 2282:	[1588090] signal (15): Terminated
      From worker 2282:	in expression starting at none:1
      From worker 2282:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2289:	
      From worker 2289:	[1588108] signal (15): Terminated
      From worker 2289:	in expression starting at none:1
      From worker 2289:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2295:	
      From worker 2295:	[1588100] signal (15): Terminated
      From worker 2295:	in expression starting at none:1
      From worker 2295:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2292:	
      From worker 2292:	[1588096] signal (15): Terminated
      F

Worker 2298 terminated.
Unhandled Task ERROR: EOFError: read end of file
Stacktrace:
 [1] (::Base.var"#wait_locked#715")(s::TCPSocket, buf::IOBuffer, nb::Int64)
   @ Base ./stream.jl:947
 [2] unsafe_read(s::TCPSocket, p::Ptr{UInt8}, nb::UInt64)
   @ Base ./stream.jl:955
 [3] unsafe_read
   @ ./io.jl:761 [inlined]
 [4] unsafe_read(s::TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
   @ Base ./io.jl:760
 [5] read!
   @ ./io.jl:762 [inlined]
 [6] deserialize_hdr_raw
   @ /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/messages.jl:167 [inlined]
 [7] message_handler_loop(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
   @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_messages.jl:172
 [8] process_tcp_streams(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
   @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_mess

      From worker 2273:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2256:	
      From worker 2256:	[319966] signal (15): Terminated
      From worker 2256:	in expression starting at none:1
      From worker 2256:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2246:	
      From worker 2246:	[319960] signal (15): Terminated
      From worker 2246:	in expression starting at none:1
      From worker 2246:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2258:	
      From worker 2258:	[319993] signal (15): Terminated
      From worker 2258:	in expression starting at none:1
      From worker 2258:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2252:	
      From worker 2252:	[319969] signal (15): Terminated
      From worker 2252:	in expression starting at none:1
      From worker 2252:	epoll_wait at /lib64/libc.so.6 (unknown line)
      From worker 2272:	
      From worker 2272:	[319951] signal (15): Terminated
      From w

Worker 2258 terminated.Unhandled Task ERROR: IOError: read: connection reset by peer (ECONNRESET)
Stacktrace:
  [1] wait_readnb(x::TCPSocket, nb::Int64)
    @ Base ./stream.jl:410
  [2] (::Base.var"#wait_locked#715")(s::TCPSocket, buf::IOBuffer, nb::Int64)
    @ Base ./stream.jl:949
  [3] unsafe_read(s::TCPSocket, p::Ptr{UInt8}, nb::UInt64)
    @ Base ./stream.jl:955
  [4] unsafe_read
    @ ./io.jl:761 [inlined]
  [5] unsafe_read(s::TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
    @ Base ./io.jl:760
  [6] read!
    @ ./io.jl:762 [inlined]
  [7] deserialize_hdr_raw
    @ /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/messages.jl:167 [inlined]
  [8] message_handler_loop(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
    @ Distributed /global/common/software/nersc/n9/julia/1.9.4/share/julia/stdlib/v1.9/Distributed/src/process_messages.jl:172
  [9] process_tcp_streams(r_stream::TCPSocket, w_stream::TCPSocket, incoming::Bool)
    